In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from lightgbm import LGBMRegressor as LGBR
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

2022-04-21 10:37:06.842631: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-21 10:37:06.842656: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def create_df(journey):
    comunio = pd.read_csv(f'../data/pruebas/only_comunio_stats_J{journey}.csv')
    clas = pd.read_excel(f'../data/classification_J_{journey}.xlsx', sheet_name=f'classification_J_{journey}', index_col='Unnamed: 0')
    cal = pd.read_csv('../data/pruebas/Season_21-22.csv')
    
    teams_dict = {'Athletic Club': 'Athletic Club' ,
                 'CA Osasuna':'Osasuna',
                 'Club Atlético de Madrid':'Atlético Madrid',
                 'Cádiz CF':'Cádiz',
                 'Deportivo Alavés':'Alavés',
                 'Elche CF':'Elche',
                 'FC Barcelona':'Barcelona',
                 'Getafe CF':'Getafe',
                 'Granada CF':'Granada',
                 'Levante UD':'Levante',
                 'RC Celta de Vigo':'Celta Vigo',
                 'RCD Espanyol de Barcelona':'Espanyol',
                 'RCD Mallorca':'Mallorca',
                 'Rayo Vallecano de Madrid':'Rayo Vallecano',
                 'Real Betis Balompié':'Betis',
                 'Real Madrid CF':'Real Madrid',
                 'Real Sociedad de Fútbol':'Real Sociedad',
                 'Sevilla FC':'Sevilla',
                 'Valencia CF':'Valencia',
                 'Villarreal CF':'Villarreal'}
    
    new_team_name = []
    for team in comunio.Team:
  
        for k,v in teams_dict.items():
       
            if k == team:
           
                new_team_name.append(v)
    
    comunio['Team'] = new_team_name
    
    points_per_team = comunio.groupby('Team').sum().reset_index()
    points_per_team= points_per_team.rename(columns={'Total_Points': 'Squad_Points',
                                                       'Points_Average': 'Squad_Average_Points',
                                                       'Avg_last_5_games': 'Squad_Avg_last_5_Games',
                                                       'Value': 'Value_Squad',
                                                       f'J_{journey -4}': f'Squad_Points_J_{journey -4}',
                                                       f'J_{journey -3}': f'Squad_Points_J_{journey -3}',
                                                       f'J_{journey -2}': f'Squad_Points_J_{journey -2}',
                                                       f'J_{journey -1}': f'Squad_Points_J_{journey -1}',
                                                       f'J_{journey}': f'Squad_Points_J_{journey}',})
    
    points_per_team = points_per_team.drop(['Team_id','On_start_%'], axis=1)
    
    df_1 = comunio.merge(points_per_team, how='left', left_on='Team', right_on='Team')
    
    matches_J = cal.loc[cal['Journey'] == (journey+1)]
    matches_J
    
    vs = []
    for team in df_1.Team:

        for index in matches_J.index:
           
            if team == matches_J.loc[index].Home:
            
                vs.append(matches_J.loc[index].Away)
                
            if team == matches_J.loc[index].Away:
                
                vs.append(matches_J.loc[index].Home)
    
    df_1['vs'] = vs
    
    points_per_vs_team = comunio.groupby('Team').sum().reset_index()
    points_per_vs_team= points_per_vs_team.rename(columns={'Total_Points': 'Vs_Squad_Points',
                                                           'Team':'Vs_Team',
                                                           'Points_Average': 'Vs_Squad_Average_Points',
                                                           'Avg_last_5_games': 'Vs_Squad_Avg_last_5_Games',
                                                           'Value':'Vs_Value_Squad',
                                                           f'J_{journey -4}': f'Vs_Squad_Points_J_{journey -4}',
                                                           f'J_{journey -3}': f'Vs_Squad_Points_J_{journey -3}',
                                                           f'J_{journey -2}': f'Vs_Squad_Points_J_{journey -2}',
                                                           f'J_{journey -1}': f'Vs_Squad_Points_J_{journey -1}',
                                                           f'J_{journey}': f'Vs_Squad_Points_J_{journey}',})
    
    points_per_vs_team = points_per_vs_team.drop(['Team_id','On_start_%'], axis=1)
    
    df_2 = df_1.merge(points_per_vs_team, how='left', left_on='vs', right_on='Vs_Team')
    
    team_clas = []

    for team in comunio.Team:
        for index in clas.index:
            
            if team in clas.loc[index].Team:
                
                team_clas.append(clas.loc[index].Position)
    
    team_clas_vs = []

    for team in df_2.Vs_Team:
        for index in clas.index:
           
            if team in clas.loc[index].Team:
                
                team_clas_vs.append(clas.loc[index].Position)
    
    df_2['Team_clas'] = team_clas
    df_2['Vs_Team_clas'] = team_clas_vs
    
    
    return df_2.drop('Team_id', axis=1)

In [3]:
def preprocess_data(train,target,journey):
    j_28_target = target[['Player',f'J_{journey}']]
    j_28_target= j_28_target.rename(columns={'Player': 'Jugador', f'J_{journey}':'Target'})
    df= train.merge(j_28_target, how='left', left_on='Player', right_on='Jugador')
    df = df.dropna()
    X = df.drop(['Target'], axis=1)._get_numeric_data()
    y = df.Target
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
    
    return X, y, X_train, X_test, y_train, y_test 

In [4]:
def preprocess_data(train,target,journey):
    j_28_target = target[['Player',f'J_{journey}']]
    j_28_target= j_28_target.rename(columns={'Player': 'Jugador', f'J_{journey}':'Target'})
    df= train.merge(j_28_target, how='left', left_on='Player', right_on='Jugador')
    df = df.dropna()
    X = df.drop(['Target'], axis=1)._get_numeric_data()
    y = df.Target
    
    x_scaler = MinMaxScaler()
    y_scaler = MinMaxScaler()
    
    X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)
    
    X_train_s = x_scaler.fit_transform(X_train)
    X_test_s = x_scaler.transform(X_test)
    
    y_train_s = y_scaler.fit_transform(y_train.values.reshape(-1,1))
    y_test_s = y_scaler.transform(y_test.values.reshape(-1,1))
                    
    return X, y, X_train, X_test, y_train, y_test ,X_train_s, X_test_s, y_train_s, y_test_s

In [5]:
j_30 = create_df(30)
j_31 = create_df(31)

In [6]:
X, y, X_train, X_test, y_train, y_test ,X_train_s, X_test_s, y_train_s, y_test_s = preprocess_data(j_30,j_31,31)

In [81]:
rfr = RandomForestRegressor(max_depth= 5, max_leaf_nodes= 30, n_estimators= 100, n_jobs= -1)
xgbr = XGBRegressor(max_depth=5, n_estimators=200,learning_rate=0.2, n_jobs=-1, gamma=0.1 )
gb = GradientBoostingRegressor(max_depth=5, n_estimators=200,learning_rate=0.005, alpha=0.9, max_leaf_nodes=30)
models = (rfr,xgbr,gb)

In [8]:
for model in models:
    name = str(model)[:13]
    if 'Sequential' in name:
        
        model.fit(X_train_s,
                   y_train_s,
                   epochs=50,
                   validation_data=0.15)
        pred = model.predict(X_test_s)
        pred2 = y_scaler.inverse_transform(pred)
        mse = mean_squared_error(pred2,y_test, squared=False)
        r2 = r2_score(pred2, y_test)
        
        pd.DataFrame(model.history).plot(figsize=(8,5))
        plt.grid()
        plt.show()
        
        
    if 'MLP' in name:
        
        model.fit(X_train_s,y_train_s)
        model.score(X_train_s, y_train_s)
        pred = model.predict(X_test_s)
        pred_test = y_scaler.inverse_transform(pred.reshape(-1,1))
        mse = mean_squared_error(pred_test,y_test, squared=False)
        
        
    else:
        '''model.fit(X_train,y_train)
        model.score(X_train, y_train)'''
        model.fit(X_train_s,y_train)
        model.score(X_train_s, y_train)
        pred = model.predict(X_test_s)
        #pred2 = y_scaler.inverse_transform(pred.reshape(-1,1))
        mse = mean_squared_error(pred,y_test, squared=False)
        r2 = r2_score(pred, y_test)
    
    print (f'mse del modelo {name} = {mse}')

mse del modelo RandomForestR = 2.8684641023573776
mse del modelo XGBRegressor( = 2.8002360929290444
mse del modelo GradientBoost = 2.933841751628109


In [82]:
rfr.fit(X,y)
xgbr.fit(X,y)
gb.fit(X,y)

new_x = j_31._get_numeric_data()
new_x

,Total_Points,Points_Average,Value,On_start_%,J_27,J_28,J_29,J_30,J_31,Avg_last_5_games,Squad_Points,Squad_Average_Points,Value_Squad,Squad_Points_J_27,Squad_Points_J_28,Squad_Points_J_29,Squad_Points_J_30,Squad_Points_J_31,Squad_Avg_last_5_Games,Vs_Squad_Points,Vs_Squad_Average_Points,Vs_Value_Squad,Vs_Squad_Points_J_27,Vs_Squad_Points_J_28,Vs_Squad_Points_J_29,Vs_Squad_Points_J_30,Vs_Squad_Points_J_31,Vs_Squad_Avg_last_5_Games,Team_clas,Vs_Team_clas
0,122,4.5,3560000,87,2,5,2,2,5,3.2,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
1,20,5.0,390000,13,0,0,0,0,0,0.0,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
2,0,0.0,180000,0,0,0,0,0,0,0.0,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
3,115,5.5,5560000,65,7,0,0,0,0,1.4,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
4,103,5.4,3390000,58,2,4,6,2,3,3.4,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
5,82,3.6,1440000,68,0,0,0,0,0,0.0,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
6,81,4.8,1750000,45,3,2,2,3,3,2.6,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
7,79,4.6,2720000,55,5,4,4,0,3,3.2,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
8,77,3.2,1200000,71,0,3,0,0,6,1.8,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12
9,45,6.4,4450000,16,5,4,13,8,0,6.0,1915,92.1,72590000,84,54,66,77,60,68.2,1823,77.7,70630000,51,42,64,62,44,52.6,8,12


In [10]:
pred_rfr = rfr.predict(X)
pred_xgb = xgbr.predict(X)
pred_gb = gb.predict(X)
preds = pd.DataFrame(j_30['Player'])
preds['Position'] = j_30.Position
preds['rfr'] = pred_rfr
preds['xgb'] = pred_xgb
preds['gb'] = pred_gb
preds = round(preds)
preds
X_train_S, X_test_S, y_train_S, y_test_S = train_test_split(preds, y, random_state=42)

In [11]:
X_train_S.shape, X_test_S.shape, y_train_S.shape, y_test_S.shape

((431, 5), (144, 5), (431,), (144,))

In [12]:
lgbr= LGBR(max_depth=5, n_estimators=150,learning_rate=0.05, n_jobs=-1)

lgbr.fit(X_train_S._get_numeric_data(), y_train_S._get_numeric_data())

train_score = lgbr.score(X_train_S._get_numeric_data(), y_train_S)
test_score = lgbr.score(X_test_S._get_numeric_data(), y_test_S)
train_lgbm_pred = lgbr.predict(X_train_S._get_numeric_data())
test_lgbm_pred = lgbr.predict(X_test_S._get_numeric_data())
train_mse = mean_squared_error(train_lgbm_pred, y_train_S, squared=False)
test_mse = mean_squared_error(test_lgbm_pred, y_test_S, squared=False)

'train R2 ', train_score, ' test R2 ', test_score, 'MSE_train', train_mse, 'MSE_test', test_mse

('train R2 ',
 0.9725939395503282,
 ' test R2 ',
 0.9877475058527837,
 'MSE_train',
 0.4961458936661032,
 'MSE_test',
 0.3560574619306217)

In [13]:
lgbr.fit(preds._get_numeric_data(), y)

LGBMRegressor(learning_rate=0.05, max_depth=5, n_estimators=150)

In [14]:
pred_lgbr = lgbr.predict(preds._get_numeric_data())


In [15]:
preds = pd.DataFrame(j_30['Player'])
preds['Position'] = j_30.Position
preds['rfr'] = pred_rfr
preds['xgb'] = pred_xgb
preds['gb'] = pred_gb
preds['lgbm'] = pred_lgbr
preds = round(preds)
preds

,Player,Position,rfr,xgb,gb,lgbm
0,Unai Simón,PT,3.0,5.0,3.0,5.0
1,Agirrezabala,PT,0.0,-0.0,1.0,-0.0
2,Ezkieta,PT,0.0,-0.0,1.0,-0.0
3,Iñigo Martínez,DF,2.0,0.0,2.0,-0.0
4,Daniel Vivian,DF,3.0,3.0,2.0,3.0
5,Lekue,DF,1.0,0.0,1.0,-0.0
6,De Marcos,DF,2.0,3.0,2.0,3.0
7,Yeray Álvarez,DF,3.0,3.0,2.0,3.0
8,Balenziaga,DF,3.0,6.0,3.0,6.0
9,Yuri Berchiche,DF,3.0,0.0,2.0,-0.0


In [16]:
pred_rfr = rfr.predict(new_x)
pred_xgb = xgbr.predict(new_x)
pred_gb = gb.predict(new_x)

In [17]:
preds = pd.DataFrame(j_30['Player'])
preds['Position'] = j_30.Position
preds['rfr'] = pred_rfr
preds['xgb'] = pred_xgb
preds['gb'] = pred_gb
preds = round(preds)
pred_lgbr = lgbr.predict(preds._get_numeric_data())
preds['lgbm'] = pred_lgbr
preds = round(preds)
preds

,Player,Position,rfr,xgb,gb,lgbm
0,Unai Simón,PT,4.0,5.0,4.0,5.0
1,Agirrezabala,PT,0.0,1.0,1.0,1.0
2,Ezkieta,PT,0.0,0.0,1.0,-0.0
3,Iñigo Martínez,DF,3.0,3.0,4.0,3.0
4,Daniel Vivian,DF,4.0,4.0,3.0,4.0
5,Lekue,DF,1.0,1.0,1.0,1.0
6,De Marcos,DF,3.0,4.0,3.0,4.0
7,Yeray Álvarez,DF,4.0,5.0,3.0,5.0
8,Balenziaga,DF,3.0,5.0,3.0,5.0
9,Yuri Berchiche,DF,4.0,2.0,3.0,2.0


In [18]:
j_32 = create_df(32)
real = j_32[['Player', 'J_32']]
real

,Player,J_32
0,Unai Simón,1
1,Agirrezabala,0
2,Ezkieta,0
3,Iñigo Martínez,0
4,Daniel Vivian,4
5,Yeray Álvarez,3
6,Lekue,0
7,De Marcos,0
8,Balenziaga,0
9,Yuri Berchiche,1


In [19]:
preds = preds.merge(real, left_on='Player',right_on='Player')
preds

,Player,Position,rfr,xgb,gb,lgbm,J_32
0,Unai Simón,PT,4.0,5.0,4.0,5.0,1
1,Agirrezabala,PT,0.0,1.0,1.0,1.0,0
2,Ezkieta,PT,0.0,0.0,1.0,-0.0,0
3,Iñigo Martínez,DF,3.0,3.0,4.0,3.0,0
4,Daniel Vivian,DF,4.0,4.0,3.0,4.0,4
5,Lekue,DF,1.0,1.0,1.0,1.0,0
6,De Marcos,DF,3.0,4.0,3.0,4.0,0
7,Yeray Álvarez,DF,4.0,5.0,3.0,5.0,3
8,Balenziaga,DF,3.0,5.0,3.0,5.0,0
9,Yuri Berchiche,DF,4.0,2.0,3.0,2.0,1


In [63]:
squad = ['Courtois', 'Bono',
         'Balliu', 
         'Fran García',
         #'Ronald Araujo', 
         'Alderete', 'Manu Sánchez', 
         #'Giménez',
         'Yéremi',  'Gavi',  'Modric',  'Brais', 'Papu Gómez', 'Dani Rodríguez', 'Pere Milla', 'Cervi', 'Berenguer',
         'Vinícius Júnior', 'Aubameyang', 'Sancet', 
         'João Félix', 
         'Morales']
df_squad_kike = pd.DataFrame()
for player in squad:
    #print (player)
    for p in preds.Player:
        if player in p:
            #print('encontrado')
            add_player = pd.DataFrame(preds.loc[preds.Player==p])
            #print(add_player)
            df_squad_kike = df_squad_kike.append(add_player)
print(len(squad),len(df_squad_kike))

df_squad_kike.at[514,'Position'] = 'MD'
round(df_squad_kike)

20 20


,Player,Position,rfr,xgb,gb,lgbm,J_32
199,Courtois,PT,5.0,4.0,3.0,4.0,0
545,Bono,PT,3.0,2.0,3.0,2.0,0
233,Balliu,DF,4.0,6.0,5.0,6.0,-4
234,Fran García,DF,4.0,3.0,4.0,3.0,4
410,Omar Alderete,DF,4.0,4.0,3.0,4.0,0
53,Manu Sánchez,DF,3.0,2.0,3.0,2.0,2
393,Yéremi Pino,MD,4.0,4.0,3.0,4.0,2
120,Gavi,MD,4.0,4.0,3.0,4.0,5
213,Modric,MD,3.0,2.0,2.0,2.0,3
528,Brais Méndez,MD,3.0,3.0,3.0,3.0,8


In [64]:
def once_ideal_xbgr(data):
    gk = data.loc[data['Position']=='PT'].sort_values('xgb', ascending=False)[:1]
    df = data.loc[data['Position']=='DF'].sort_values('xgb', ascending=False)[:3]
    md = data.loc[data['Position']=='MD'].sort_values('xgb', ascending=False)[:4]
    dl = data.loc[data['Position']=='DL'].sort_values('xgb', ascending=False)[:3]
    positions = [gk, df, md, dl]
    squad = pd.DataFrame()
    for pos in positions:
        squad = squad.append(pd.DataFrame(pos))
    
    return squad

In [65]:
xgb_eleven = round(once_ideal_xbgr(df_squad_kike))
xgb_eleven

,Player,Position,rfr,xgb,gb,lgbm,J_32
199,Courtois,PT,5.0,4.0,3.0,4.0,0
233,Balliu,DF,4.0,6.0,5.0,6.0,-4
410,Omar Alderete,DF,4.0,4.0,3.0,4.0,0
234,Fran García,DF,4.0,3.0,4.0,3.0,4
297,Dani Rodríguez,MD,4.0,5.0,4.0,5.0,2
393,Yéremi Pino,MD,4.0,4.0,3.0,4.0,2
120,Gavi,MD,4.0,4.0,3.0,4.0,5
514,Pere Milla,MD,3.0,4.0,3.0,4.0,2
46,João Félix,DL,4.0,7.0,3.0,7.0,1
221,Vinícius Júnior,DL,6.0,4.0,3.0,5.0,1


In [66]:
print('Totales xgb: ', sum(xgb_eleven.xgb), ', real: ', sum(xgb_eleven.J_32))

Totales xgb:  49.0 , real:  14


In [67]:
def once_ideal_rf(data):
    gk = data.loc[data['Position']=='PT'].sort_values('rfr', ascending=False)[:1]
    df = data.loc[data['Position']=='DF'].sort_values('rfr', ascending=False)[:3]
    md = data.loc[data['Position']=='MD'].sort_values('rfr', ascending=False)[:4]
    dl = data.loc[data['Position']=='DL'].sort_values('rfr', ascending=False)[:3]
    positions = [gk, df, md, dl]
    squad = pd.DataFrame()
    for pos in positions:
        squad = squad.append(pd.DataFrame(pos))
    
    return squad

In [68]:
def once_ideal_gb(data):
    gk = data.loc[data['Position']=='PT'].sort_values('gb', ascending=False)[:1]
    df = data.loc[data['Position']=='DF'].sort_values('gb', ascending=False)[:3]
    md = data.loc[data['Position']=='MD'].sort_values('gb', ascending=False)[:4]
    dl = data.loc[data['Position']=='DL'].sort_values('gb', ascending=False)[:3]
    positions = [gk, df, md, dl]
    squad = pd.DataFrame()
    for pos in positions:
        squad = squad.append(pd.DataFrame(pos))
    
    return squad

In [69]:
rf_eleven = round(once_ideal_rf(df_squad_kike))
rf_eleven

,Player,Position,rfr,xgb,gb,lgbm,J_32
199,Courtois,PT,5.0,4.0,3.0,4.0,0
233,Balliu,DF,4.0,6.0,5.0,6.0,-4
234,Fran García,DF,4.0,3.0,4.0,3.0,4
410,Omar Alderete,DF,4.0,4.0,3.0,4.0,0
393,Yéremi Pino,MD,4.0,4.0,3.0,4.0,2
120,Gavi,MD,4.0,4.0,3.0,4.0,5
297,Dani Rodríguez,MD,4.0,5.0,4.0,5.0,2
14,Álex Berenguer,MD,4.0,3.0,3.0,3.0,4
221,Vinícius Júnior,DL,6.0,4.0,3.0,5.0,1
129,Aubameyang,DL,6.0,4.0,4.0,5.0,1


In [70]:
print('Totales rfr: ', sum(rf_eleven.rfr), ', real: ', sum(rf_eleven.J_32))

Totales rfr:  49.0 , real:  16


In [71]:
gb_eleven = round(once_ideal_gb(df_squad_kike))
gb_eleven

,Player,Position,rfr,xgb,gb,lgbm,J_32
199,Courtois,PT,5.0,4.0,3.0,4.0,0
233,Balliu,DF,4.0,6.0,5.0,6.0,-4
234,Fran García,DF,4.0,3.0,4.0,3.0,4
410,Omar Alderete,DF,4.0,4.0,3.0,4.0,0
297,Dani Rodríguez,MD,4.0,5.0,4.0,5.0,2
393,Yéremi Pino,MD,4.0,4.0,3.0,4.0,2
120,Gavi,MD,4.0,4.0,3.0,4.0,5
528,Brais Méndez,MD,3.0,3.0,3.0,3.0,8
129,Aubameyang,DL,6.0,4.0,4.0,5.0,1
221,Vinícius Júnior,DL,6.0,4.0,3.0,5.0,1


In [72]:
print('Totales gb: ', sum(gb_eleven.gb), ', real: ', sum(gb_eleven.J_32))

Totales gb:  38.0 , real:  21


In [73]:
def once_ideal_lgbm(data):
    gk = data.loc[data['Position']=='PT'].sort_values('lgbm', ascending=False)[:1]
    df = data.loc[data['Position']=='DF'].sort_values('lgbm', ascending=False)[:3]
    md = data.loc[data['Position']=='MD'].sort_values('lgbm', ascending=False)[:4]
    dl = data.loc[data['Position']=='DL'].sort_values('lgbm', ascending=False)[:3]
    positions = [gk, df, md, dl]
    squad = pd.DataFrame()
    for pos in positions:
        squad = squad.append(pd.DataFrame(pos))
    
    return squad

In [74]:
lgbm_eleven = round(once_ideal_lgbm(df_squad_kike))
lgbm_eleven

,Player,Position,rfr,xgb,gb,lgbm,J_32
199,Courtois,PT,5.0,4.0,3.0,4.0,0
233,Balliu,DF,4.0,6.0,5.0,6.0,-4
410,Omar Alderete,DF,4.0,4.0,3.0,4.0,0
234,Fran García,DF,4.0,3.0,4.0,3.0,4
297,Dani Rodríguez,MD,4.0,5.0,4.0,5.0,2
393,Yéremi Pino,MD,4.0,4.0,3.0,4.0,2
120,Gavi,MD,4.0,4.0,3.0,4.0,5
514,Pere Milla,MD,3.0,4.0,3.0,4.0,2
46,João Félix,DL,4.0,7.0,3.0,7.0,1
221,Vinícius Júnior,DL,6.0,4.0,3.0,5.0,1


In [75]:
print('Totales lgbm: ', sum(lgbm_eleven.lgbm), ', real: ', sum(lgbm_eleven.J_32))

Totales lgbm:  51.0 , real:  14


In [76]:
import pickle

In [83]:
pickle.dump(rfr, open( 'comunio_rfr.model', 'wb') )
pickle.dump(xgbr, open( 'comunio_xgbr.model', 'wb') )
pickle.dump(gb, open( 'comunio_gb.model', 'wb') )

In [78]:
comunio_rfr = pickle.load(open('comunio_rfr.model', 'rb'))

In [79]:
comunio_rfr.predict(new_x)

array([4.1017356 , 0.23560055, 0.07164884, 3.13490845, 3.8909884 ,
       0.86409864, 3.01021784, 3.64866928, 3.3135559 , 4.14207564,
       1.28028493, 1.46339109, 0.08676288, 5.77644241, 4.2166616 ,
       3.42061835, 2.83699075, 1.05115684, 3.60978838, 1.56911915,
       0.4630193 , 4.13703401, 3.38924677, 3.66814278, 1.54939163,
       3.99820518, 0.05714287, 3.3195952 , 3.15970943, 0.93372913,
       2.94016425, 2.91261981, 2.65320321, 3.35975231, 3.80652456,
       3.74062425, 2.84223756, 3.25255416, 3.23354917, 4.57625559,
       1.17409198, 1.99366116, 0.05714287, 3.41774651, 2.28135146,
       3.54072572, 4.44749202, 3.21862236, 2.67594878, 4.08225495,
       0.07260608, 4.23171733, 3.28941191, 3.19615955, 1.74284358,
       3.95274053, 1.69655263, 2.79509185, 0.07260608, 0.07260608,
       0.07260608, 4.05847907, 3.97776297, 3.54576301, 3.24042229,
       2.2963919 , 2.0533742 , 0.85537223, 0.49622755, 1.17785633,
       0.16850352, 0.07260608, 3.72492749, 3.1953303 , 4.89815